# Interactive atom-based property visualisation in the browser

### Paolo Tosco (Novartis), RDKit UGM 2025

In [ ]:
import os
import re
import itertools
import pandas as pd
import json
import requests
import requests.utils
import time
import uuid
import logging
import urllib.request
import urllib.parse
from io import BytesIO, StringIO
from xml.dom import minidom
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole, InteractiveRenderer, rdMolDraw2D, rdDepictor
from rdkit.Chem import PandasTools
from IPython.display import SVG, HTML, display
import ipywidgets

In [ ]:
DATA_ROOT_URL = "https://ptosco.github.io/ugm2025"

Load the drawing settings used internally at Novartis Biomedical Research:

In [ ]:
rdkit_drawing_settings_json_url = f"{DATA_ROOT_URL}/rdkit_drawing_settings.json"
response = urllib.request.urlopen(rdkit_drawing_settings_json_url)
assert response.status == 200
rdkit_drawing_settings_json = response.read().decode("utf-8")

In [ ]:
rdMolDraw2D.UpdateMolDrawOptionsFromJSON(IPythonConsole.drawOptions, rdkit_drawing_settings_json)
IPythonConsole.drawOptions.standardColoursForHighlightedAtoms = True

Load the interactive `rdkit-structure-renderer`

In [ ]:
InteractiveRenderer.rdkitStructureRendererJsUrl = f"{DATA_ROOT_URL}/rdkit-structure-renderer-module.js"

In [ ]:
InteractiveRenderer.minimalLibJsUrl = f"{DATA_ROOT_URL}/RDKit_minimal.js"

In [ ]:
InteractiveRenderer.setEnabled()

I took a table of 106 compounds whose pKa was experimentally assessed from this publication:<br><br>
Settimo, L.; Bellman, K.; Knegtel, R. M. A.<br>
[Comparison of the Accuracy of Experimental and Predicted pKa Values of Basic and Acidic Compounds](https://dx/doi.org/10.1007/s11095-013-1232-z)<br>
<i>Pharm. Res.</i> <b>2014</b>, <i>31</i>, 1082–1095<br><br>
and, more specifically, from a [piece of Supplementary Information](https://static-content.springer.com/esm/art%3A10.1007%2Fs11095-013-1232-z/MediaObjects/11095_2013_1232_MOESM1_ESM.docxhttps://static-content.springer.com/esm/art%3A10.1007%2Fs11095-013-1232-z/MediaObjects/11095_2013_1232_MOESM1_ESM.docx)
which I converted to `xlsx` format for easier manipulation with `pandas`.

<b>TL;DR</b> If you are interested in how I processed the information in the table and fetched PubChem identifiers, keep reading below.<br>
Otherwise feel free to skip to the <b>Read pickled DataFrame</b> section.

In [ ]:
response = urllib.request.urlopen(f"{DATA_ROOT_URL}/Vertex_106_compounds.xlsx")
assert response.status == 200
with BytesIO(response.read()) as hnd:
    df = pd.read_excel(hnd)

Our table had one `Exp. pka` value per row, labelled as `acid` or `base`:

In [ ]:
df.head()

We'd rather have all values on the same row, so we pivot the table:

In [ ]:
df = df.pivot(index=["Smiles string", "name"], columns="type", values="Exp. pka")
df.columns.name = None
df = df.rename(columns={t: f"Exp pKa {t[0].upper()}" for t in df.columns}).reset_index()

In [ ]:
df.head()

We also miss common names for these molecules, so we'll fetch them from PubChem:

In [ ]:
smi_list = df["Smiles string"].apply(lambda smi: urllib.parse.urlencode((("smiles", smi),))).to_list()

In [ ]:
PUG_REST_API_BASE_URL = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"

In [ ]:
client = requests.session()

In [ ]:
client.headers = {
    "accept": "application/json",
    "Content-Type": "application/x-www-form-urlencoded"
}

First we get PubChem CIDs from SMILES...

In [ ]:
cid_status_list = []
for smi in smi_list:
    response = client.post(f"{PUG_REST_API_BASE_URL}/compound/smiles/cids/TXT", data=smi)
    cid = None
    if response.status_code == 200:
        cid = response.text.strip()
    cid_status_list.append((cid, response.status_code))
    # PubChem usage policy is no more than 5 requests/second
    time.sleep(0.2)

In [ ]:
assert all(cid for cid, _ in cid_status_list)

In [ ]:
cids = [cid for cid, _ in cid_status_list]

In [ ]:
len(cids)

...then we get PubChem titles (i.e., common names):

In [ ]:
titles = []
for cid in cids:
    cid_encoded =  urllib.parse.urlencode([("cid", cid)])
    response = client.post(f"{PUG_REST_API_BASE_URL}/compound/cid/description/JSON", data=cid_encoded)
    title = None
    if response.status_code == 200:
        response_json = response.json()
        cid_title_list = [i for i in response_json["InformationList"]["Information"] if "CID" in i and "Title" in i]
        assert len(cid_title_list) == 1
        title = cid_title_list[0]["Title"]
    titles.append(title)
    time.sleep(0.2)

In [ ]:
assert all(titles)
assert len(titles) == len(cids)

In [ ]:
df["PubChem CID"] = cids
df["PubChem title"] = titles

Finally we save the pickled `DataFrame` so we do not have to do this again:

In [ ]:
df.to_pickle("Vertex_106_compounds.pkl")

### Read pickled DataFrame

In [ ]:
if os.path.exists("Vertex_106_compounds.pkl"):
    df = pd.read_pickle("Vertex_106_compounds.pkl")
else:
    response = urllib.request.urlopen(f"{DATA_ROOT_URL}/Vertex_106_compounds.pkl")
    assert response.status == 200
    with BytesIO(response.read()) as hnd:
        df = pd.read_pickle(hnd)

We canonicalize SMILES with RDKit and generate a `ROMol` column:

In [ ]:
df["ROMol"] = df["Smiles string"].apply(Chem.MolFromSmiles)
assert all(df["ROMol"])
df["Smiles string"] = df["ROMol"].apply(Chem.MolToSmiles)
df["ROMol"] = df["Smiles string"].apply(Chem.MolFromSmiles)
assert all(df["ROMol"])

Then we save the molecules as SDF file; we'll need the SDF file to predict pKa values with [MoKa](https://www.moldiscovery.com/software/moka/):

In [ ]:
PandasTools.WriteSDF(df, "Vertex_106_compounds.sdf", "ROMol", "PubChem title", properties=list(df.columns))

<b>NOTE:</b> The following is unlikely to work in your environment, unless you are licensing MoKa and have a `MoKa` module available to be loaded.<br>
In that case , skip directly to the <b>Parse MoKa predictions from text output</b> section.<br>
We compute a maximum of 3 acidic and 3 basic pKa values:

In [ ]:
MAX_MOKA_PKA = 3

In [ ]:
!module load MoKa && echo \
    moka_cli --input-type=sd --output-type=txt --sd-field=\"PubChem title\" --acid-values={MAX_MOKA_PKA}

In [ ]:
!module load MoKa && \
    moka_cli --input-type=sd --output-type=txt --sd-field="PubChem title" \
    --acid-values={MAX_MOKA_PKA} --basic-values={MAX_MOKA_PKA} \
    --output=./Vertex_106_compounds_moka.txt \
    ./Vertex_106_compounds.sdf

### Parse MoKa predictions from text output

In [ ]:
if os.path.exists("Vertex_106_compounds_moka.txt"):
    with open("Vertex_106_compounds_moka.txt") as hnd:
        moka_results = [line for line in hnd]
else:
    response = urllib.request.urlopen(f"{DATA_ROOT_URL}/Vertex_106_compounds_moka.txt")
    assert response.status == 200
    with StringIO(response.read().decode("utf-8")) as hnd:
        moka_results = [line for line in hnd]

Make sure that we have as many non-null results as entries in the `DataFrame`:

In [ ]:
assert all(moka_results)
assert len(moka_results) == len(df)

This is how MoKa results look like:<br>
`<NUM_PREDICTIONS> <PKA_TYPE_PRED_1> <PKA_VALUE_PRED_1> <ATOM_IDX_PRED_1> <SD_PRED_1> {...repeat for NUM_PREDICTIONS...}`

In [ ]:
moka_results[:3]

We define a few classes to load MoKa-predicted and experimental pKa on the molecules for interactive visualization:

In [ ]:
class MokaPrediction:

    """Store predicted and experimental pKa values."""

    def __init__(self, pka_type=None, pka=None, sd=None, exp_pka=None):
        self.pka_type = pka_type
        self.pka = pka
        self.sd = sd
        self.exp_pka = exp_pka

In [ ]:
class MokaAtom:

    """Manage atom-related pKa prediction.

    Organize pKa prediction by pKa type (acid, base).
    Create atom highlights, labels and tooltips.
    """

    STRIP_RGB_HEX_PREFIX = re.compile(r"(?i)^(?:0x|#)?([0-9a-f]{6})")
    ACID_BASE_RGB = {
        "A": "#febfb8",
        "B": "#b1d5e7"
    }
    HIGHLIGHT_RADIUS = 0.4
    TOOLTIP_CSS = """\
.rdk-str-rnr-tooltip .atom {
    table {
        border-collapse: collapse;
    }
    th, td {
        white-space: nowrap;
        text-align: center;
        padding-left:5px;
        padding-right:5px;
        font-size:1.2em;
    }
}"""

    def __init__(self):
        self.prediction_by_pka_type = {}

    @staticmethod
    def float_triplet_to_hexcode(r, g, b):
        """Convert R, G, B float triplet to #RRGGBB hex code."""
        return "#" + "".join(f"{round(c * 255):02x}" for c in (r, g, b))

    @classmethod
    def hexcode_to_float_triplet(cls, h):
        """Convert #RRGGBB hex code to R, G, B float triplet."""
        m = cls.STRIP_RGB_HEX_PREFIX.match(h)
        if not m:
            return None
        h = m[1]
        return tuple(float(int(f"0x{h[i*2:(i+1)*2]}", 16)) / 255 for i in range(3))

    def add_prediction(self, prediction):
        """Add MokaPrediction to this atom."""
        self.prediction_by_pka_type[prediction.pka_type] = prediction

    def pka_highlight_colors(self):
        """Return list of R, G, B tuples to highlight this atom."""
        return [self.hexcode_to_float_triplet(self.ACID_BASE_RGB[pka_type])
                for pka_type, prediction in sorted(self.prediction_by_pka_type.items())]

    def pka_label(self):
        """Return pKa label for this atom."""
        pka_values = [f"{prediction.pka:.2f}" for pka_type, prediction in sorted(self.prediction_by_pka_type.items())]
        return ",".join(pka_values)
        
    def pka_tooltip(self):
        """Return pKa HTML tooltip for this atom."""
        thead = ["<thead><tr><th></th><th>MoKa &plusmn; SD</th>"]
        exp_pka_type_tuple = None
        tbody = {}
        for pka_type, prediction in sorted(self.prediction_by_pka_type.items()):
            tbody[pka_type] = [f"<tr style=\"background-color: {self.ACID_BASE_RGB[pka_type]};\">"
                               f"<td><b>{pka_type}</b></td><td>{prediction.pka:.2f} &plusmn; {prediction.sd:.2f}</td>"]
            if prediction.exp_pka is not None:
                exp_pka_type_tuple = (prediction.exp_pka, pka_type)
        if exp_pka_type_tuple is not None:
            thead.append("<th>Exp</th>")
            exp_pka, exp_pka_type = exp_pka_type_tuple
            for pka_type, trow in tbody.items():
                exp_pka_td_content = ""
                if exp_pka_type == pka_type:
                    exp_pka_td_content = f"{exp_pka:.2f}"
                trow.append(f"<td>{exp_pka_td_content}</td>")
        thead.append("</thead>")
        for trow in tbody.values():
            trow.append("</tr>")
        tooltip = [f"<style>{self.TOOLTIP_CSS}</style>"] + ["<table>"] + thead + ["".join(trow) for _, trow in sorted(tbody.items())] + ["</table>"]
        tooltip_encoded = "".join(tooltip)
        return tooltip_encoded

    def pka_highlight_radius(self):
        """Return highlight radius for this atom."""
        return self.HIGHLIGHT_RADIUS

In [ ]:
class MokaMol:

    """Store collection of MokaAtoms by pKa type.

    Also store a reference to ROMol such that pka
    visualization can be set up on the molecule.
    """

    N_MOKA_FIELDS = 4
    MOKA_VALUE_PARSE_REGEX = re.compile("^(\"[^\"]+\"|[^\"^ ]+) 0 - (\\d+)(.*)$")

    @classmethod
    def from_moka_result(cls, moka_result):
        """Return MokaPrediction array from MoKa raw result."""
        if not cls.MOKA_VALUE_PARSE_REGEX.match(moka_result):
            raise ValueError(f"MoKa result '{moka_result}' cannot be parsed")
        moka_result_parsed = cls.MOKA_VALUE_PARSE_REGEX.sub(r"\2,\3", moka_result)
        n_pred, pka_values = moka_result_parsed.split(",")
        n_pred = int(n_pred)
        pka_value_iter = iter(pka_values.strip().split())
        pka_values_sliced = [tuple(itertools.islice(pka_value_iter, cls.N_MOKA_FIELDS)) for i in range(n_pred)]
        if tuple(itertools.islice(pka_value_iter, cls.N_MOKA_FIELDS)):
            raise ValueError(f"Found more predictions in MoKa result '{moka_result}' than expected ({n_pred})")
        moka_mol = cls()
        for pka_type, pka, atom_idx, sd in pka_values_sliced:
            pka_type = pka_type.upper()
            if not pka_type in ("A", "B"):
                raise ValueError(f"Expected pKa type in MoKa result '{moka_result}' to be 'A' or 'B', found {pka_type}")
            pka = float(pka)
            if (pka_type == "A" and pka > 14) or (pka_type == "B" and pka < 0):
                continue
            atom_idx = int(atom_idx)
            if atom_idx <= 0:
                raise ValueError(f"Expected atom idx in MoKa result '{moka_result}' to be >0, found {atom_idx}")
            atom_idx -= 1
            sd = float(sd)
            moka_mol.add_atom_prediction(atom_idx, MokaPrediction(pka_type=pka_type, pka=pka, sd=sd))
        return moka_mol

    def __init__(self):
        self.atoms = {}

    def set_romol(self, mol):
        """Set RDKit ROMol."""
        self.mol = mol

    def add_atom_prediction(self, atom_idx, prediction):
        """Add MokaPrediction to atom with index atom_idx."""
        moka_atom = self.atoms.get(atom_idx, MokaAtom())
        moka_atom.add_prediction(prediction)
        self.atoms[atom_idx] = moka_atom

    def set_exp_pka(self, exp_pka, pka_type):
        """Associate experimental pKa to closest MokaPrediction."""
        pkas_by_type = [atom for atom in self.atoms.values()
                        if pka_type in atom.prediction_by_pka_type.keys()]
        if exp_pka is not None and str(exp_pka) != "nan" and pkas_by_type:
            closest_atom = min(pkas_by_type,
                key=lambda atom: abs(atom.prediction_by_pka_type[pka_type].pka - exp_pka))
            closest_atom.prediction_by_pka_type[pka_type].exp_pka = exp_pka
            return True
        return False

    def setup_mol_for_pka_visualization(self):
        """Add opts to ROMol for interactive visualization."""
        if self.mol is None:
            return False
        highlight_atom_multiple_colors = {}
        highlight_atom_radii = {}
        atom_tooltips = {}
        for atom_idx, moka_atom in self.atoms.items():
            pka_label = moka_atom.pka_label()
            atom_tooltips[atom_idx] = moka_atom.pka_tooltip()
            atom = self.mol.GetAtomWithIdx(atom_idx)
            atom.SetProp("atomNote", pka_label)
            highlight_atom_multiple_colors[atom_idx] = moka_atom.pka_highlight_colors()
            highlight_atom_radii[atom_idx] = moka_atom.pka_highlight_radius()
        atom_tooltips_json = json.dumps(atom_tooltips)
        opts = {
            "data-draw-opts": {
                "highlightAtomMultipleColors": highlight_atom_multiple_colors,
                "highlightAtomRadii": highlight_atom_radii
            },
            "data-atom-tooltips": atom_tooltips_json
        }
        InteractiveRenderer.setOpts(self.mol, opts)
        return True

Now we:
* parse the predictions generated by MoKa for each `DataFrame` molecule
* associate the closest prediction to the experimentally measured pKa value
* prepare the RDKit molecule for interactive visualization:

In [ ]:
for mol_idx, moka_result in enumerate(moka_results):
    moka_mol = MokaMol.from_moka_result(moka_result)
    moka_mol.set_romol(df.loc[mol_idx].ROMol)
    for pka_type in ("A", "B"):
        exp_pka = df.iloc[mol_idx][f"Exp pKa {pka_type}"]
        moka_mol.set_exp_pka(exp_pka, pka_type)
    moka_mol.setup_mol_for_pka_visualization()

We call the familiar `PandasTools.RenderImagesInAllDataFrames()` to render molecules in the `DataFrame` as images, but since earlier we enabled interactive rendering, we will have interactive HTML5 objects rather than static images:

In [ ]:
PandasTools.RenderImagesInAllDataFrames()

We drop a few columns that we do not need anymore, since their information is more usefully visualized on the molecule itself:

In [ ]:
df_show = df.drop(columns=["Smiles string", "name", "Exp pKa A", "Exp pKa B"]).reset_index(drop=True).set_index("PubChem title")

Let's make the molecules a bit larger than the default:

In [ ]:
IPythonConsole.molSize = (300, 200)
PandasTools.molSize = (300, 200)

And, finally, let's display the `DataFrame`.<br>
You should be able to see pKa tooltips hovering on the highlighted atoms.

In [ ]:
df_show.head()

Here's another example of interactivity applied to atoms and bonds: selecting atoms and bonds of an RDKit molecule in Jupyter.<br>Through a simple helper class it is possible to access the selection from Python.

In [ ]:
class AtomBondSelection:

    """A class to enable interactive atom and bond selection in a Jupyter Notebook."""

    ATOM_SELECTION_ATTR = "data-atom-selection"
    BOND_SELECTION_ATTR = "data-bond-selection"
    DRAW_OPTS_ATTR = "data-draw-opts"
    RETURN_DRAW_COORDS_ATTR = "returnDrawCoords"
    JP_WIDGET_CLASSNAME = "jp-atom-selection"
    STANDARD_COLOR_OPT_NAME = "standardColoursForHighlightedAtoms"
    DRAWING_EXTENTS_INCLUDE_OPT_NAME = "drawingExtentsInclude"
    RDK_STR_RNR_PREFIX = "rdk-str-rnr-"
    MOL_ID_REGEX = re.compile(f"^{RDK_STR_RNR_PREFIX}mol-([0-9a-f]{{8}}-(?:[0-9a-f]{{4}}-){{3}}[0-9a-f]{{12}})$")

    def __init__(self, mol):
        self._widget = ipywidgets.Text(disabled=False, layout=ipywidgets.Layout(display="None"))
        self._widget_classname = f"{self.JP_WIDGET_CLASSNAME}-{str(uuid.uuid1())}"
        self._widget.observe(self.handler, names="value")
        self._widget.add_class(self._widget_classname)
        self._mol = Chem.Mol(mol)
        self._orig_repr_html_ = self._mol._repr_html_
        self._logger = logging.getLogger(self.__class__.__name__)
        self._atoms = None

    def _repr_html_(self):
        """The HTML representation of this class."""
        display(self._widget)
        self._mol_repr_dom = minidom.parseString(self._orig_repr_html_().replace(" scoped", ""))
        divs = [div for div in self._mol_repr_dom.getElementsByTagName("div") if self.MOL_ID_REGEX.match(div.getAttribute("id"))]
        if len(divs) != 1:
            raise ValueError(f"Failed to find unique mol div; found {len(divs)}")
        self._mol_div = divs[0]
        self._mol_div_id = self._mol_div.getAttribute("id")
        self._mol_div_uuid = self.MOL_ID_REGEX.match(self._mol_div.getAttribute("id"))[1]
        self._mol_div.parentNode.insertBefore(self._jupyter_comm_script(), self._mol_div)
        self._mol_div.setAttribute(self.ATOM_SELECTION_ATTR, "")
        self._mol_div.setAttribute(self.BOND_SELECTION_ATTR, "")
        draw_opts = json.loads(self._mol_div.getAttribute(self.DRAW_OPTS_ATTR) or "{}")
        draw_opts[self.STANDARD_COLOR_OPT_NAME] = True
        draw_opts[self.DRAWING_EXTENTS_INCLUDE_OPT_NAME] = { "ALL": True, "HIGHLIGHTS": False }
        self._mol_div.setAttribute(self.DRAW_OPTS_ATTR, json.dumps(draw_opts))
        return self._mol_repr_dom.toxml()
        
    def _jupyter_comm_script(self):
        """The script which syncs the HTML widget with Jupyter."""
        script = self._mol_repr_dom.createElement("script")
        script_text = self._mol_repr_dom.createTextNode("""\
(function() {
    const tid = setInterval(function() {
        try {
            const jpAtomSel = document.getElementsByClassName('""" + self._widget_classname + """');
            if (jpAtomSel.length !== 1) return;
            const input = jpAtomSel[0].querySelector('input');
            if (!input) return;
            const molDiv = document.getElementById('""" + self._mol_div_id + """');
            if (!molDiv) return;
            const r = RDKitStructureRenderer;
            const divId = r.getDivId(molDiv);
            if (!r.setUserOptsCallback(divId, function({divId, atoms, bonds}) {
                input.value = JSON.stringify({atoms, bonds});
                input.dispatchEvent(new Event('input', { bubbles: true, cancelable: true }));
            })) return;
        } catch(e) {
            const msgDiv = document.createElement('div');
            msgDiv.className = 'lm-Widget p-Widget jp-RenderedText jp-mod-trusted jp-OutputArea-output';
            msgDiv.textContent = 'Failed to set up Jupyter communication (${e})';
            molDiv.parentNode.appendChild(msgDiv)
        }
        if (tid) {
            clearInterval(tid);
        }
    }, 10);
})();
""")
        script.appendChild(script_text)
        return script

    def handler(self, obj):
        """The Jupyter widget handler callback."""
        self._obj = obj
        if obj.get("type", None) == "change" and obj.get("name", None) == "value":
            new_value = obj.get("new", None)
            if new_value is not None:
                try:
                    self._selection = json.loads(new_value)
                except Exception as e:
                    self._logger.warning(f"Failed to parse JSON '{new_value}'")

    def atoms(self):
        """Selected atom accessor."""
        return self._selection["atoms"]

    def bonds(self):
        """Selected bond accessor."""
        return self._selection["bonds"]

Instantiate the `AtomBondSelection` class:

In [ ]:
atom_bond_selection = AtomBondSelection(df_show.loc["Cefdinir"].ROMol)

Visualize it in Jupyter.<br>
Atoms and bonds can be selected by mouse clicks...

In [ ]:
atom_bond_selection

...and selections can be accessed from Python:

In [ ]:
atom_bond_selection.atoms()

In [ ]:
atom_bond_selection.bonds()